In [1]:
from multiprocessing.sharedctypes import Value
import networkx as nx
import numpy as np
import random
import pandas as pd

from tqdm import tqdm
from datetime import datetime
import pickle

# File imports
import data
from data import select_dataset, get_station_g, ingest_electricity_data, set_random_speed_columns, ingest_pems
from replicate_graph import layer_graph
from vehicle import Vehicle
from simulation import Simulation
from visualization import set_draw_attributes
from vehicle_list_data import get_utilization

## Ingest pems for the whole graph (all charging and parking stops)
### commented out because it only needs to be done once. Csv already saved.

In [2]:
# all_graph = ingest_pems("data/charging_parking_stations_lonlat.csv", "data/charging_parking_distances.csv")

In [3]:
# #save the new csv with all the speeds (from pems ingest above), which will be used for the algorithm
# all_graph[1].to_csv("data/charging_parking_distances_pemsingested.csv")

## Run the algorithm

In [4]:
number_of_iterations = 5
kwh_per_km = 1.9

#scenario list: each list inside has three params : charging rate, kwh per km, and battery capacity.
# scenario_list = [[45,1.9,215],[90,1.9,215],[45,0.95,215],[45,1.9,430],[90,0.95,430]] 
scenario_list = [[90,0.95,430], [45,1.9,430], [45,0.95,215], [90,1.9,215], [45,1.9,215]] 

columns = ["scenario_number", "iteration", "charging_rate", "km_per_kwh", "battery_capacity", "success",
           "station_utilization_disp_of_avg", "station_utilization_avg_of_disp", "electricity",
           "percent_delay", "hours_spent_in_queues", "hours_spent_charging"]
results = [columns]

def print_and_record_results(sim, scenario, s, n, success=True):
    print("name: ", sim.name)
    print("success: ", success)
    
    if success==True:
        result = [s, n, scenario[0], scenario[1], scenario[2], True,
           sim.metrics['station_utilization_disp_of_avg'], sim.metrics['station_utilization_avg_of_disp'], 
           sim.metrics['electricity'], sim.metrics['percent_delay'],
           sim.metrics['hours_spent_in_queues'], sim.metrics['hours_spent_charging']]
        print("station_utilization_disp_of_avg:  ", sim.metrics['station_utilization_disp_of_avg'])
        print("station_utilization_avg_of_disp:  ", sim.metrics['station_utilization_avg_of_disp'])
        print("electricity:  ", sim.metrics['electricity'])
        print("percent_delay:  ", sim.metrics['percent_delay'])
        print("hours_spent_in_queues:  ", sim.metrics['hours_spent_in_queues'])
        print("hours_spent_charging:  ", sim.metrics['hours_spent_charging'])
        print("________________________________________")
        results.append(result)
    else:
        result = [s, n, scenario[0], scenario[1], scenario[2], False, None, None, None, None, None, None]
        results.append(result)

for scenario, s in zip(scenario_list, range(0,len(scenario_list))):
    
    print('------- SCENARIO '+str(s)+' -------')
    #initialize for wcctici
    simulation_length = 12
    battery_interval = 20
    kwh_per_km = scenario[1]
    battery_capacity = scenario[2]
    stations_path = "data/wcctci_stations-updated.csv"
    distances_path = "data/wcctci_coord_distances.csv"
    name = "anna_reverse_debugged_scenario"+str(s)+"_wcctci" 
    
    for n in range(0, number_of_iterations):
        
        ### Print metrics
        sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
        sim.add_demand_nodes()
        try:
            sim.run()
            print_and_record_results(sim, scenario, s, n, True)
        except Exception as e: 
            print(e)
            print_and_record_results(sim, scenario, s, n, False)
            continue

        ### get station utilization

        #get the station ids in order 
        current_stations = [int(j) for j in sim.station_g.nodes]

#         #get the utilization rate of the stations
#         utilization = sim.metrics['station_utilization']
        
        # Alternative utilization calculation 
        utilization = get_utilization(sim)
        
        avg_utilization = {}
        for node in current_stations:
            avg_utilization[node] = 0
            for i in range(len(utilization)):
                use_i = utilization[i]
                try:
                    avg_utilization[node] += use_i[str(node)]
                except:
                    avg_utilization[node] += 0

            avg_utilization[node] = avg_utilization[node]/len(utilization)

        utilization = [i for i in avg_utilization.values()]

        #get the station that has the highest utilization.
        h_st = current_stations[np.argmax(utilization)]

        #get the station that has the lowest utilization.
        l_st = current_stations[np.argmin(utilization)]

        print("Highest:", h_st)
        print("Lowest:", l_st)
        ### Find the unused location that is closest to the highly utilized station

        #get the map of all parking areas and stations
        full_map_dist = pd.read_csv("data/charging_parking_distances_pemsingested.csv")
        full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

        #keep only the roads that start or end with the high utilization station
        h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

        #then we eliminate all of the stations that are already being used.
        using_st = current_stations.copy()
        using_st.remove(h_st)
        h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

        #then we remove the case where we go from h_st to h_st
        h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

        #find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

        new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
        new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

        #one of the two will be h_st and the other will be the new station location. 
        if new_origin == h_st:
            new_station = new_destination
        else:
            new_station = new_origin

        print("New Station:", new_station)
        ### create the new maps with the locations and the distances of the stations that we will be using next

        new_stations_list = current_stations + [new_station]
        new_stations_list.remove(l_st)

        #now we reduce these to the new set of stations. All origins and destinations should be in this list.
        new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
        new_map_loc = full_map_loc[full_map_loc.loc[:, 'OID_'].isin(new_stations_list)]

        #we add the charging rates and capacity- assuming that they are the same as the others (set to 45 and 8)
        #TO DO: check this
        new_map_loc['charging_rate'] = scenario[0]
        new_map_loc['physical_capacity'] = 8

        #we change the snapX and snapY columns to lon and lat 
        new_map_loc = new_map_loc.rename(columns = {'SnapX': 'longitude', 'SnapY': 'latitude'})

        #now we save them. 
        new_map_loc.to_csv("data/algorithm_"+str(n+1)+"_stations.csv") 
        new_map_dist.to_csv("data/algorithm_"+str(n+1)+"_distances.csv")


        #reset the variables for the next iteration
        stations_path = "data/algorithm_"+str(n+1)+"_stations.csv"
        distances_path = "data/algorithm_"+str(n+1)+"_distances.csv"
        name = "anna_reverse_debugged_scenario"+str(s)+"_algorithm_"+str(n+1)

    #we run the simulation for the final alocation

    ### run the simulation

    sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
    sim.add_demand_nodes()
    
    try:
        sim.run()
        print_and_record_results(sim, scenario, s, n, True)
    except:
        print_and_record_results(sim, scenario, s, n, False)
        

pd.DataFrame(results).to_csv("algorithm_run_reverse_trial.csv")

------- SCENARIO 0 -------
Strongly connected components: 147
Weakly connected components: 44
Weakly connected?: False


100%|██████████| 12/12 [01:35<00:00,  7.98s/it]


name:  anna_reverse_debugged_scenario0_wcctci
success:  True
station_utilization_disp_of_avg:   4.7125
station_utilization_avg_of_disp:   5.209635416666667
electricity:   0
percent_delay:   (1.0, 1.0, 1.49150552377569, 1.0112326078099392, 0.05032649267273029)
hours_spent_in_queues:   (0.0, 0.0, 2.8, 0.03707465718638903, 0.22694735981425304)
hours_spent_charging:   (0, 0.0, 11, 0.5063484002031488, 1.7014573631022256)
________________________________________
Highest: 33
Lowest: 1
New Station: 212


/Users/alandler/.local/lib/python3.7/site-packages/ipykernel_launcher.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/alandler/.local/lib/python3.7/site-packages/ipykernel_launcher.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Strongly connected components: 147
Weakly connected components: 44
Weakly connected?: False


...

name:  anna_reverse_debugged_scenario4_algorithm_3
success:  True
station_utilization_disp_of_avg:   15.959114583333333
station_utilization_avg_of_disp:   16.57265625
electricity:   0
percent_delay:   (1.0, 1.4576853573060293, 3.88216057277714, 1.780828985677216, 0.8494049051935947)
hours_spent_in_queues:   (0.0, 0.0, 5.0, 0.9399568034557236, 1.3214060833108603)
hours_spent_charging:   (0, 4.0, 17, 2.587473002159827, 3.2605394296621024)
________________________________________
Highest: 247
Lowest: 4
New Station: 62
Strongly connected components: 274
Weakly connected components: 45
Weakly connected?: False


100%|██████████| 12/12 [01:22<00:00,  6.91s/it]


name:  anna_reverse_debugged_scenario4_algorithm_4
success:  True
station_utilization_disp_of_avg:   16.592968749999997
station_utilization_avg_of_disp:   17.341666666666665
electricity:   0
percent_delay:   (1.0, 1.5713507080288427, 3.88216057277714, 1.7648939510782897, 0.7139559180310854)
hours_spent_in_queues:   (0.0, 0.0, 4.4, 0.8272340425531916, 1.1263681083184347)
hours_spent_charging:   (0, 4.0, 16, 2.676595744680851, 2.647697736650229)
________________________________________
Highest: 247
Lowest: 5
New Station: 69
Strongly connected components: 272
Weakly connected components: 45
Weakly connected?: False


100%|██████████| 12/12 [01:07<00:00,  5.62s/it]


name:  anna_reverse_debugged_scenario4_algorithm_5
success:  True
station_utilization_disp_of_avg:   16.257291666666667
station_utilization_avg_of_disp:   17.411458333333332
electricity:   0
percent_delay:   (1.0, 1.6106330948671457, 4.0398830607824205, 1.8417655945172926, 0.8100369901937212)
hours_spent_in_queues:   (0.0, 0.4, 4.6, 1.0601018675721563, 1.2924177431851274)
hours_spent_charging:   (0, 2.0, 16, 2.2275042444821733, 2.4031368970737104)
________________________________________


In [5]:
utilization

[0.0,
 1.94375,
 0.0,
 13.085416666666667,
 0.0,
 0.0,
 0.0,
 0.0,
 5.039583333333334,
 6.00625,
 0.9229166666666667,
 15.78125,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 22.81875,
 7.945833333333334,
 5.8,
 11.91875,
 0.0,
 4.81875,
 0.0,
 0.0,
 0.0,
 7.627083333333333,
 13.739583333333334,
 15.964583333333334,
 24.13125,
 14.497916666666667]

In [6]:
# avg_utilization = {}
# for node in current_stations:
#     avg_utilization[node] = 0
#     for i in range(len(utilization)):
#         use_i = utilization[i]
#         try:
#             avg_utilization[node] += use_i[str(node)]
#         except:
#             avg_utilization[node] += 0
    
#     avg_utilization[node] = avg_utilization[node]/len(utilization)

# utilization_list = [i for i in avg_utilization.values()]

## Bellow this is  just code I used to think about/ build the algorithm above (Just ignore).
### I was thinking about just one iteration, starting from the wcctci pickle.

In [7]:
#open the pickle
with open('trials/wcctci_04_04_2022_12_15_28.pkl', 'rb') as inp:
    res = pickle.load(inp)

Get the highest and lowest utilized stations:

In [8]:
#get station utilization

#get the station ids in order 
current_stations = list(res.station_g.nodes)

#get the utilization rate of the stations
utilization = res.metrics['station_utilization']

#get the station that has the highest utilization.
h_st = current_stations[np.argmax(utilization)]

#get the station that has the lowest utilization.
l_st = current_stations[np.argmin(utilization)]

TypeError: 'NoneType' object is not subscriptable

Find the unused location that is closest to the highly utilized station:

In [ ]:
#get the map of all parking areas and stations
full_map_dist = pd.read_csv("data/charging_parking_distances.csv")

#keep only the roads that start or end with the high utilization station
h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

#then we eliminate all of the stations that are already being used.
using_st = current_stations.copy()
using_st.remove(h_st)
h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

#then we remove the case where we go from h_st to h_st
h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

#find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

#one of the two will be h_st and the other will be the new station location. 
if new_origin == h_st:
    new_station = new_destination
else:
    new_station = new_origin

Now we start to think about re-runing the simulation with this swap (taking out l_st and adding new_station). To do this, we need to create a new map with the locations and the distances of the stations that we will be using next.

In [ ]:
new_stations_list = current_stations + [new_station]

In [ ]:
#we have "full_map_dist" (distances map)
#and we need the locations map
full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

#now we reduce these to the new set of stations. All origins and destinations should be in this list.
new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
new_map_loc = full_map_loc[(full_map_loc.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_loc.loc[:, 'DestinationID'].isin(new_stations_list))]

In [ ]:
#now we save them. 
new_map_loc.to_csv("data/algorithm_"+str(1)+"_stations.csv") ########change the 1
new_map_dist.to_csv("data/algorithm_"+str(1)+"_distances.csv")

Now we can run the next simulation

In [ ]:
simulation_length = 12
battery_interval = 20
km_per_percent = 3.13

sim = Simulation("data/algorithm_"+str(1)+"_stations.csv", "data/algorithm_"+str(1)+"_distances.csv", simulation_length, battery_interval, km_per_percent)

# Anna

In [ ]:
stations_path = "data/wcctci_stations-updated.csv"
distances_path = "data/wcctci_coord_distances.csv"
# stations_df, distances_df = select_dataset(stations_csv_path, distances_csv_path)
# station_g = get_station_g(stations_df, distances_df)
# battery_g = layer_graph(station_g, 20, km_per_percent= 1.9)
simulation_length = 24
battery_interval = 20
kwh_per_km = 1.9
battery_capacity = 215
name = 'hi'
sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
sim.add_demand_nodes()

In [ ]:
sim.battery_g.nodes

In [ ]:
nx.shortest_path(sim.battery_g, "Long Beach", "Reno")